<a href="https://colab.research.google.com/github/lumenintellects/seq2seq-chatbot/blob/main/CSK507_EOM_CTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CSK507 End of Module Generative Chatbot


##1. Import and initialise


###1.1 Import
First, import everything needed during pre-processing and training


In [ ]:
import datetime
overallStart = datetime.datetime.now()
start = datetime.datetime.now()
print(start)
import locale
print('Prefferred encoding: ',locale.getpreferredencoding())
locale.getpreferredencoding = lambda: "UTF-8"
import os #needed to manipulate the downloaded files within the collab environment and to pull data
import json
from google.colab import userdata
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from matplotlib import pyplot as plt
%matplotlib inline
import spacy
from spacy import displacy
import numpy as np
import pandas as pd
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import zipfile
import spacy.cli
from spacy.lang.en import English # updated
gpu = spacy.prefer_gpu()
from collections import Counter
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
from ipywidgets import IntProgress
from IPython.display import display
from google.colab import drive
drive.mount('/content/drive')

end = datetime.datetime.now()


print(end)

print('Time taken to import all packages: ',end-start)



runtype = 'ubuntu' #'ubuntu' or 'wiki'

2024-12-09 12:38:19.859721
Prefferred encoding:  UTF-8
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
2024-12-09 12:38:44.129259
Time taken to import all packages:  0:00:24.269538


##1.2 Download and unzip dataset
We need to pull the dataset down and unzip from https://www.microsoft.com/en-us/download/confirmation.aspx?id=52419

In [ ]:
#https://www.microsoft.com/en-us/download/confirmation.aspx?id=52419
start = datetime.datetime.now()
print(start)

if runtype == 'wiki':
  if os.path.exists('content/WikiQACorpus.zip') == False:
    print('zip doesn''t exist, downloading')
    !wget -P content https://download.microsoft.com/download/E/5/F/E5FCFCEE-7005-4814-853D-DAA7C66507E0/WikiQACorpus.zip
  else:
    print('WikiQACorpus.tsv already exists, not need to download')

  if os.path.exists('content/WikiQACorpus/WikiQA.tsv') == False:
    print('zip not unzipped, unzipping')
    with zipfile.ZipFile('content/WikiQACorpus.zip', 'r') as zip_ref:
      zip_ref.extractall('content')
  else:
    print('WikiQA.tsv already exists, not need to unzip')
else
  print('Not running wikiQA corpus')

end = datetime.datetime.now()

print(end)

print('Time taken to download and unzip: ',end-start)

2024-12-09 12:38:44.141563
zip doesnt exist, downloading
--2024-12-09 12:38:44--  https://download.microsoft.com/download/E/5/F/E5FCFCEE-7005-4814-853D-DAA7C66507E0/WikiQACorpus.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.200.181.207, 2600:1417:3f:59b::317f, 2600:1417:3f:580::317f
Connecting to download.microsoft.com (download.microsoft.com)|23.200.181.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7094233 (6.8M) [application/octet-stream]
Saving to: ‘content/WikiQACorpus.zip’

WikiQACorpus.zip    100%[===================>]   6.76M  --.-KB/s    in 0.1s    

2024-12-09 12:38:44 (56.1 MB/s) - ‘content/WikiQACorpus.zip’ saved [7094233/7094233]

zip not unzipped, unzipping
2024-12-09 12:38:44.590409
Time taken to download and unzip:  0:00:00.448846


##1.3 Pandas DF
Put data into pandas df

In [ ]:
start = datetime.datetime.now()
print(start)

if runtype == 'wiki':
  df = pd.read_csv('content/WikiQACorpus/WikiQA.tsv',on_bad_lines='skip',engine='python',sep='\t')
  end = datetime.datetime.now()
  #df = df.loc[0:1000]
else
  print('Not running wikiQA corpus')


print(end)

print('Time taken to load to df: ',end-start)

2024-12-09 12:38:44.597653
2024-12-09 12:38:44.785008
Time taken to load to df:  0:00:00.187355


#2. Text normalisation, cleaning and extraction

##2.1 Normalise using NFKD
We first have to nomalise the data using NFKD to make standard alpha so that _é_ becomes _e_ etc. Additionally, check the coding by converting to ascii and back to UTF-8.

In [ ]:
start = datetime.datetime.now()
print(start)

if runtype == 'wiki':
  df['Question'] = df.Question.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8') # this will make the text normal alpha, for exaple é will be e, etc
  df['Sentence'] = df.Sentence.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8') # this will make the text normal alpha, for exaple é will be e, etc
else
  print('Not running wikiQA corpus')

end = datetime.datetime.now()

print(end)

print('Time taken to nomalise: ',end-start)


2024-12-09 12:38:44.792687
2024-12-09 12:38:44.871904
Time taken to nomalise:  0:00:00.079217


##2.2 Clean data
Next we clean the data by removing triple and double spaces befre making lower case

In [ ]:
start = datetime.datetime.now()
print(start)

if runtype == 'wiki':
  df['Question'] = df['Question'].str.replace('\n',' ') # remove carriage returns
  df['Question'] = df['Question'].str.replace('   ',' ') #remove triple spaces
  df['Question'] = df['Question'].str.replace('  ',' ') # also remove double sapces (which might remain after triples are removed)
  df['Question'] = df['Question'].str.lower()
  df['Sentence'] = df['Sentence'].str.replace('\n',' ') # remove carriage returns
  df['Sentence'] = df['Sentence'].str.replace('   ',' ') #remove triple spaces
  df['Sentence'] = df['Sentence'].str.replace('  ',' ') # also rmeove double sapces (which might remain after triples are removed)
  df['Sentence'] = df['Sentence'].str.lower()
else
  print('Not running wikiQA corpus')

end = datetime.datetime.now()

print(end)

print('Time taken to clean: ',end-start)

2024-12-09 12:38:44.879578
2024-12-09 12:38:44.949762
Time taken to clean:  0:00:00.070184


##2.3 Data Extraction
Next we extract the relevant normalised and cleansed columns of dataframe into ques and ans dataframes

In [ ]:
start = datetime.datetime.now()
print(start)

if runtype == 'wiki':
  ques = df['Question']
  ans = df['Sentence']
else
  print('Not running wikiQA corpus')

end = datetime.datetime.now()
print(end)

print('Time taken to extract: ',end-start)

2024-12-09 12:38:44.956053
2024-12-09 12:38:44.956418
Time taken to extract:  0:00:00.000365


#2.4 Export to csv for team wide review

In [ ]:
if runtype == 'wiki':
  outputdf = pd.DataFrame({'input': ques, 'output': ans})
  outputdf.to_csv('outputdf.csv')
else
  print('Not running wikiQA corpus')

##2.5 Switch to Ubuntu

In [ ]:
start = datetime.datetime.now()
print(start)

if runtype == 'ubuntu':
  dfu = pd.read_csv('/content/drive/MyDrive/ubuntu_dialogue_corpus_input_output_pairs.csv',on_bad_lines='skip',engine='python')



  print('length of df before splitting:', len(dfu))

  first_PCT = int(len(dfu)*0.005)
  df = dfu.loc[0:first_PCT]

  print('length of df after splitting:', len(df))


  ques = df['input']
  ans = df['output']
  df = pd.DataFrame({'input': ques, 'output': ans})
else
  print('Not running ubuntu corpus')

#df = df.loc[0:100]
end = datetime.datetime.now()

print(end)

print('Time taken to load to df: ',end-start)

2024-12-09 12:38:45.101486
length of df before splitting: 7636876
length of df after splitting: 38185
2024-12-09 12:39:18.801219
Time taken to load to df:  0:00:33.699733


In [ ]:
df.head()

,input,output
0,sock puppe?t wtf?,"it's a wikipedia term. there is prodigy, prdig..."
1,probably not a vector icon,the other icons aren't vector icons either.
2,how do fix vlc in the terminal?,fix what? check other programs. same faint sound?
3,"ok, anyone know how i set bios password in ubu...",you'd need to set that in your bios.
4,how do i generate an xorg.conf file? for intel...,"'sudo xorg -configure', i think"


#3. Tokenization and indexing

Next we need to tokenize the data using spaCy before making word-to-index and index-to-word mappings. Finally we will need to convert toindices for the model

##3.1 Import spaCy Pipeline

Large Class used because of copus complexity, en_core_web_lg contains 343k unique vectors, compared with en_core_web_sm which only has 20k

In [ ]:
start = datetime.datetime.now()
print(start)

#because of the complex corpus, use the larger en_core_web_lg

try:
  nlp = spacy.load("en_core_web_lg")
except OSError:
  print('Downloading en_core_web_lg')
  spacy.cli.download("en_core_web_lg")
  nlp = spacy.load("en_core_web_lg")

end = datetime.datetime.now()

print(end)

print('Time taken to download and load spaCy class: ',end-start)

2024-12-09 12:39:18.903391
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


2024-12-09 12:39:37.926194
Time taken to download and load spaCy class:  0:00:19.022803


##3.2 Counters

We need a way of getting and storing the frequency, so we need to initalise counters (from collections package)

In [ ]:
start = datetime.datetime.now()
print(start)




ques_words = Counter()
ans_words = Counter()


end = datetime.datetime.now()
print(end)

print('Time taken to initialise counters: ',end-start)

2024-12-09 12:39:37.934617
2024-12-09 12:39:37.934863
Time taken to initialise counters:  0:00:00.000246


##3.3 Tokenize

totokenize, we use spaCy pieline already loaded and go through the entire df row by row before stitching _EOS onto the end for the seq-to-seq model later

In [ ]:
#start = datetime.datetime.now()
#print(start)
#longString = ' '.join(df.input.astype(str).tolist()) #get all comments into string
#print('cp1')
#lowerString = ' '.join(longString.lower().split())
#print('cp2 ')
#words = lowerString.split()
#print('cp3')
#

In [ ]:
##print(longString)
#len(longString)
#dfw = pd.DataFrame({'word': list(words)})
#dfw.drop_duplicates(inplace=True)
#dfw

In [ ]:
#ques = ques.apply(lambda x: str(x) + ' _EOS')
#ques

In [ ]:
#print(dfw)
#terms = np.unique(words)
#print('cp4')
#
#terms = np.sort(terms, axis=None)
#end = datetime.datetime.now()
#print(end)
#
#print('Time taken to get BoW: ',end-start)


In [ ]:
start = datetime.datetime.now()
print(start)

import os
import pandas as pd
from ipywidgets import IntProgress
from IPython.display import display
import spacy

# ... (rest of your imports and code)

start = datetime.datetime.now()
print(start)

# Check if the CSV files already exist
csv_files_exist = all([
    os.path.exists('/content/drive/MyDrive/ques_inputs.csv'),
    os.path.exists('/content/drive/MyDrive/ans_inputs.csv'),
    os.path.exists('/content/drive/MyDrive/ques_words.csv'),
    os.path.exists('/content/drive/MyDrive/ans_words.csv')
])

if csv_files_exist:
    print("CSV files already exist. Skipping tokenization.")
    # Load the data from the CSV files
    ques_inputs = pd.read_csv('/content/drive/MyDrive/ques_inputs.csv').values.tolist()
    ans_inputs = pd.read_csv('/content/drive/MyDrive/ans_inputs.csv').values.tolist()
    ques_words = pd.read_csv('/content/drive/MyDrive/ques_words.csv').values.tolist()
    ans_words = pd.read_csv('/content/drive/MyDrive/ans_words.csv').values.tolist()
else:
    print("CSV files not found. Starting tokenization.")
    gpu = spacy.prefer_gpu() #GPU was found to be slightly better peforming in colab for SpaCy. The GPU selection needs to be immediately before the load command so need to reload
    nlp = spacy.load("en_core_web_lg") # large was used

    ques_inputs = []
    ans_inputs = []

    f = IntProgress(min=0, max=len(df)) # instantiate the bar  [[McAteer, S (2017) Stackoverflow: How do I implement a progress bar, avaialable at https://stackoverflow.com/a/41457700 (accessed 25/11/2024)]]
    display(f) # display the bar

    for i in range(len(df)):
        ques_tokens = nlp(str(ques[i]))
        ans_tokens = nlp(str(ans[i]))




        if (len(ques_tokens)!=0 and len(ans_tokens)!=0):


          for token in ques_tokens:
              ques_words.update([token.text]) #this is the counter for the question frequency, update it


          ques_inputs.append([token.text for token in ques_tokens] + ['_EOS'])

          for token in ans_tokens:
                ans_words.update([token.text]) #this is the counter for the answer frequency, update it

          ans_inputs.append([token.text for token in ans_tokens] + ['_EOS'])
        f.value += 1
    pd.DataFrame(ques_inputs).to_csv('/content/drive/MyDrive/ques_inputs.csv', index=False)
    pd.DataFrame(ans_inputs).to_csv('/content/drive/MyDrive/ans_inputs.csv', index=False)
    pd.DataFrame(ques_words).to_csv('/content/drive/MyDrive/ques_words.csv', index=False)
    pd.DataFrame(ans_words).to_csv('/content/drive/MyDrive/ans_words.csv', index=False)
end = datetime.datetime.now()

print(end)

print('Time taken to tokenize: ',end-start)



2024-12-09 12:49:12.891145
2024-12-09 12:49:12.891400
CSV files already exist. Skipping tokenization.
2024-12-09 12:49:15.011198
Time taken to tokenize:  0:00:02.119798


In [34]:
for i in range(len(ques_inputs)):
    ques_inputs[i] = [int(x) for x in ques_inputs[i] if x != '' and not pd.isnull(x)]

for i in range(len(ans_inputs)):
    ans_inputs[i] = [int(x) for x in ans_inputs[i] if x != '' and not pd.isnull(x)]

print(ques_inputs[1])
print(ans_inputs[1])

[352, 40, 7, 4961, 564, 1]
[5, 185, 1322, 37, 20, 3167, 1322, 333, 7, 1]


##3.4 Vocabulary creation

Next we need to create word-to-index and index-to-word for both questions and answers

In [ ]:

start = datetime.datetime.now()
print(start)


#ques_words = ['_SOS','_EOS','_UNK'] + sorted(ques_words,key=ques_words.get,reverse=True) #add 'special words' like EOS to the entire word list
#ans_words = ['_SOS','_EOS','_UNK'] + sorted(ans_words,key=ans_words.get,reverse=True)

ques_words = [item for sublist in ques_words for item in sublist]
ans_words = [item for sublist in ans_words for item in sublist]

ques_w2i = {a:i for i,a in enumerate(ques_words)} #record the index against each word with the word first in a dictionary
ans_w2i = {a:i for i,a in enumerate(ans_words)}


ques_i2w = {i:o for i,o in enumerate(ques_words)} #do the same thing but in reverse so the index is first
ans_i2w = {i:o for i,o in enumerate(ans_words)}

g = IntProgress(min=0, max=len(ques_inputs)) # instantiate the bar  [[McAteer, S (2017) Stackoverflow: How do I implement a progress bar, avaialable at https://stackoverflow.com/a/41457700 (accessed 25/11/2024)]]
display(g) # display the bar

#for i in range(len(ques_inputs)):
for i in range(len(ans_inputs)):
    ques_sentence = str(ques_inputs[i])
    ans_sentence = str(ans_inputs[i])
    ques_inputs[i] = [ques_w2i.get(word, ques_w2i['_UNK']) for word in ques_inputs[i]]  # Use get to handle missing words, replace with _UNK token
    ans_inputs[i] = [ans_w2i.get(word, ans_w2i['_UNK']) for word in ans_inputs[i]]  # Use get to handle missing words, replace with _UNK token
    g.value += 1

end = datetime.datetime.now()

print(end)

print('Time taken to create vocabulary: ',end-start)

2024-12-09 12:39:44.131860


IntProgress(value=0, max=38185)

2024-12-09 12:39:55.508576
Time taken to create vocabulary:  0:00:11.376716


#lifted straight from CSK507 coursework:

In [ ]:
class EncoderLSTM(nn.Module):
  def __init__(self, vocab_len, input_dim, hidden_dim, n_layers=1, drop_prob=0):
    super(EncoderLSTM, self).__init__()

    self.input_dim = input_dim
    self.hidden_dim = hidden_dim
    self.n_layers = n_layers

    self.embedding = nn.Embedding(vocab_len, input_dim)
    self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers,
                        dropout=drop_prob, batch_first=True)

  def forward(self, inputs, encoder_state_vector, encoder_cell_vector):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    embedded = self.embedding(inputs).to(device)
    # Pass the embedded word vectors into LSTM and return all outputs
    output, hidden = self.lstm(embedded, (encoder_state_vector, encoder_cell_vector))
    return output, hidden

  def init_hidden(self, batch_size=1):
    return (torch.zeros(self.n_layers, batch_size,
                        self.hidden_dim),
            torch.zeros(self.n_layers, batch_size,
                        self.hidden_dim))

class DecoderLSTM(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_vocab_len, n_layers=1, drop_prob=0.1):
    super(DecoderLSTM, self).__init__()
    self.hidden_dim = hidden_dim
    self.output_vocab_len = output_vocab_len
    self.n_layers = n_layers
    self.drop_prob = drop_prob
    self.input_dim = input_dim

    self.embedding = nn.Embedding(self.output_vocab_len, self.input_dim)
    self.dropout = nn.Dropout(self.drop_prob)
    self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, batch_first=True)
    self.classifier = nn.Linear(self.hidden_dim, self.output_vocab_len)

  def forward(self, inputs, decoder_state_vector, decoder_context_vector):
    # Embed input words
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    embedded = self.embedding(inputs).view(1, -1).to(device)
    embedded = self.dropout(embedded)
    embedded = embedded.unsqueeze(0)

    output, hidden = self.lstm(embedded, (decoder_state_vector,
                                          decoder_context_vector))

    # Pass LSTM outputs through a Linear layer acting as a classifier
    output = F.log_softmax(self.classifier(output[0]), dim=1)

    return output, hidden



In [ ]:

start = datetime.datetime.now()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(start)
print()
#i=0
input_dim = 10000
hidden_dim = 256
spacy.prefer_gpu()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder = EncoderLSTM(len(ques_words), input_dim, hidden_dim).to(device)
decoder = DecoderLSTM(input_dim, hidden_dim, len(ans_words)).to(device)

lr = 0.001
#encoder_optimizer = optim.Adam(encoder.parameters(), lr=lr)
#decoder_optimizer = optim.Adam(decoder.parameters(), lr=lr)


if os.path.exists("/content/drive/MyDrive/model_enc_dec.pt"):
    # Load the existing model
    checkpoint = torch.load("/content/drive/MyDrive/model_enc_dec.pt", map_location=torch.device(device))
    encoder.load_state_dict(checkpoint['encoder'])
    decoder.load_state_dict(checkpoint['decoder'])
    ##encoder_optimizer.load_state_dict(checkpoint['q_optimizer'])
    #decoder_optimizer.load_state_dict(checkpoint['a_optimizer'])
    print("Existing model loaded.")
else:
    # Create a new model
    encoder = EncoderLSTM(len(ques_words), input_dim, hidden_dim).to(device)
    decoder = DecoderLSTM(input_dim, hidden_dim, len(ans_words)).to(device)
    encoder_optimizer = optim.Adam(encoder.parameters(), lr=lr)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=lr)
    print("New model created.")

EPOCHS = 10
teacher_forcing_prob = 0.5
encoder.train()
decoder.train()
tk0 = range(1,EPOCHS+1)
tk2 = enumerate(ques_inputs)
max_index = 0
for b, sentence in tk2:
  for bb in range(len(ans_inputs[b])):
    max_index += 1

k = IntProgress(min=0, max=max_index * EPOCHS) # instantiate the bar  [[McAteer, S (2017) Stackoverflow: How do I implement a progress bar, avaialable at https://stackoverflow.com/a/41457700 (accessed 25/11/2024)]]
print('overall training progress')
print()
display(k)
print()
for epoch in tk0:
    avg_loss = 0.
    j = IntProgress(min=0, max=max_index) # instantiate the bar  [[McAteer, S (2017) Stackoverflow: How do I implement a progress bar, avaialable at https://stackoverflow.com/a/41457700 (accessed 25/11/2024)]]
    print('epoch ', epoch)
    display(j) # display the bar
    print()
    tk1 = enumerate(ques_inputs)
    for i, sentence in tk1:
        #print(i)
        #print(sentence)
        loss = 0.

        #initialise encoder state vector and cell state vector
        h = encoder.init_hidden()
        encoder_state_vector = h[0]
        encoder_state_vector = encoder_state_vector.to(device)
        encoder_cell_vector = h[0]
        encoder_cell_vector = encoder_cell_vector.to(device)

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        inp = torch.tensor(sentence).unsqueeze(0).to(device)

        #print('inp: ', epoch, inp)
        #if (i % 1000) == 0:
        #  print('inp: ', i, epoch)
        encoder_outputs, h = encoder(inp, encoder_state_vector, encoder_cell_vector)

        #First decoder input will be the SOS token
        decoder_input = torch.tensor([ques_w2i['_SOS']]).to(device)
        #First decoder hidden state will be last encoder hidden state
        decoder_hidden = h

        output = []
        teacher_forcing = True if random.random() < teacher_forcing_prob else False
        #k = IntProgress(min=0, max=len(ans_inputs[i])) # instantiate the bar  [[McAteer, S (2017) Stackoverflow: How do I implement a progress bar, avaialable at https://stackoverflow.com/a/41457700 (accessed 25/11/2024)]]
        #k.value=0
        #display(k) # display the bar

        for ii in range(len(ans_inputs[i])):
          decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden[0], decoder_hidden[1])

          # Get the index value of the word with the highest score from the decoder output
          top_value, top_index = decoder_output.topk(1)
          if teacher_forcing:
            decoder_input = torch.tensor([ans_inputs[i][ii]]).to(device)
          else:
            decoder_input = torch.tensor([top_index.item()]).to(device)

          output.append(top_index.item())
          # Calculate the loss of the prediction against the actual word
          loss += F.nll_loss(decoder_output.view(1,-1), torch.tensor([ans_inputs[i][ii]]).to(device))
          j.value += 1
          k.value += 1
          #print(j.value)
          #print((j.value/(len(ans_inputs) * max_index))*100)

        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()
        avg_loss += loss.item()/len(ques_inputs)
    print(avg_loss)
    torch.save({"encoder":encoder.state_dict(),"decoder":decoder.state_dict(),"q_optimizer":encoder_optimizer.state_dict(),"a_optimizer":decoder_optimizer},"/content/drive/MyDrive/model_enc_dec.pt")



end = datetime.datetime.now()
print(end)

print('Time taken to train model: ',end-start)

2024-12-09 13:20:07.853206



<ipython-input-36-c75561ac8a1f>:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("/content/drive/MyDrive/model_enc_dec.pt", map_location=torch.devic

Existing model loaded.
overall training progress



IntProgress(value=0, max=4012060)


epoch  1


IntProgress(value=0, max=401206)

In [35]:

start = datetime.datetime.now()
print(start)
from google.colab import drive
drive.mount('/content/drive')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpoint = torch.load("/content/drive/MyDrive/model_enc_dec.pt", map_location=torch.device(device))

if os.path.exists("/content/drive/MyDrive/model_enc_dec.pt"):
    # Load the existing model
    #checkpoint = torch.load("/content/drive/MyDrive/model_enc_dec.pt")
    input_dim = 10000
    hidden_dim = 256
    lr = 0.001
    encoder = EncoderLSTM(len(ques_words), input_dim, hidden_dim).to(device)
    decoder = DecoderLSTM(input_dim, hidden_dim, len(ans_words)).to(device)
    encoder_optimizer = optim.Adam(encoder.parameters(), lr=lr)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=lr)
    encoder.load_state_dict(checkpoint['encoder'])
    decoder.load_state_dict(checkpoint['decoder'])
    ##encoder_optimizer.load_state_dict(checkpoint['q_optimizer'])
    #decoder_optimizer.load_state_dict(checkpoint['a_optimizer'])
    print("Existing model loaded.")


encoder.load_state_dict(checkpoint['encoder'])
decoder.load_state_dict(checkpoint['decoder'])
encoder_optimizer.load_state_dict(checkpoint['q_optimizer'])
#decoder_optimizer.load_state_dict(checkpoint['a_optimizer'])

encoder.eval()
decoder.eval()

# get some random numbers to choose random sentences
#rand_integers = [random.randint(0, 1000) for i in range(1, 20)] #original line that is incorrect
rand_integers = [random.randint(0, len(ques_inputs) - 1) for i in range(1, 20)]

for i in rand_integers:
  h = encoder.init_hidden()
  inp = torch.tensor(ques_inputs[i]).unsqueeze(0).to(device)
  encoder_outputs, h = encoder(inp, h[0].to(device), h[1].to(device))
  print('inp:',inp)
  print('en_inputs[',i,']: ', ques_inputs[i])
  decoder_input = torch.tensor([ques_w2i['_SOS']]).to(device)
  decoder_hidden = h
  output = []
  attentions = []
  while True:
    decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden[0], decoder_hidden[1])
    _, top_index = decoder_output.topk(1)
    decoder_input = torch.tensor([top_index.item()]).to(device)
    # If the decoder output is the End Of Sentence token, stop decoding process
    if top_index.item() == ans_w2i["_EOS"]:
      break
    output.append(top_index.item())

  print("Question: "+ " ".join([ques_i2w[x] for x in ques_inputs[i]]))
  print("Answer Predicted: " + " ".join([ans_i2w[x] for x in output]))
  print("Actual: " + " ".join([ans_i2w[x] for x in ans_inputs[i]]))
  print()



end = datetime.datetime.now()
print(end)

print('Time taken to test model: ',end-start)



2024-12-09 13:18:58.992081
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-35-bc17726db70c>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("/content/drive/MyDrive/model_enc_dec.pt", map_location=torch.device

Existing model loaded.
inp: tensor([[179, 265, 391,   5,  95, 142,  26,  39,   6, 373, 128,  15, 265,   1]],
       device='cuda:0')
en_inputs[ 25538 ]:  [179, 265, 391, 5, 95, 142, 26, 39, 6, 373, 128, 15, 265, 1]
Question: well dapper seems to run ok anyone know the xorg version in dapper _EOS
Answer Predicted: i is a lot
Actual: 7.0 _EOS

inp: tensor([[ 59,  16,   4,  30,   7, 106,  16,  12,  14,   4, 146,  49, 779, 551,
           5, 356, 551, 335,   6, 375,   3,   1]], device='cuda:0')
en_inputs[ 13523 ]:  [59, 16, 4, 30, 7, 106, 16, 12, 14, 4, 146, 49, 779, 551, 5, 356, 551, 335, 6, 375, 3, 1]
Question: hello . i have a question . how do i upgrade from 6.06 lts to 8.04 lts via the shell ? _EOS
Answer Predicted: i
Actual: you can try , but i would not ... it will probably get messed up i have played it before . _EOS

inp: tensor([[ 88, 365,   1]], device='cuda:0')
en_inputs[ 3943 ]:  [88, 365, 1]
Question: # hardware _EOS
Answer Predicted: thanks
Actual: & ompaul : thanks _EOS

in